# My little Alexa

In this Notebook you can find a basic ChatBot with limited capabilities. This project will help you understand the fundamentals of chatbot development and natural language processing (NLP).

## What Will You Learn?

- How to capture audio input
- Basic NLP techniques for understanding user intent
- Generating appropriate responses
- Converting text to speech

## How to Start?

To get a grasp of how the code is structured, navigate to the end of this notebook and look for the main function running the chatbot. This will serve as your entry point into understanding how everything comes together.

Feel free to run cells, inspect variables, and even modify the code to see how it affects the bot's behavior.

Let's get started!

**Hint:** Remember what you have learned throughout these months. Most of the code is already known ;)

In [1]:
import speech_recognition as sr

import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import cess_esp
from nltk.tag.hmm import HiddenMarkovModelTagger

from nltk.chunk.regexp import *

from datetime import datetime
import wikipedia

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from lat_lon_parser import parse
import requests
import pyttsx3
from deep_translator import GoogleTranslator

# import openai

In [2]:
# You can find your API key at https://platform.openai.com/account/api-keys
# However, it is not a Free platform, so I will comment the sections working with openai
# in case you want to see how it would work.

# api_key = 'PASTE_HERE_YOUR_API_KEY'
# openai.api_key = api_key

In [3]:
traductor_es = GoogleTranslator(source='en', target='es')
traductor_en = GoogleTranslator(source='es', target='en')

In [4]:
hmm = HiddenMarkovModelTagger.train(cess_esp.tagged_sents())

In [5]:
reglas_weather = r'''
    Lugar: {<np0000l>}
    Lugar: <sp.*> {<da0.* | vmn.*>}
    Fecha: {<rg>}
    Fecha: <sp.*> {<nc.*>}
'''
parser = nltk.RegexpParser(reglas_weather)

reglas_knowledge = r'''
    Elemento: <vm.*> <sp.*> {<.*>+}
'''
parser_knowledge = nltk.RegexpParser(reglas_knowledge)

In [6]:
mic = sr.Microphone()
instance = sr.Recognizer()
def audio_input():
    mic = sr.Microphone()
    
    with mic as source:
        instance.adjust_for_ambient_noise(source)
        audio = instance.listen(source)
        transcript = instance.recognize_google(audio, language='es-ES', show_all=True)
        
        return transcript['alternative'][0]['transcript']

In [7]:
def extrae_tiempo(_tree):
    result = {}
    
    for branch in _tree:
        if type(branch) != tuple:
            word, tag = branch[0]
            result[branch.label()] = word
            
    return result

In [8]:
def extrae_knowledge(_tree):
    
    element = ''
    
    for branch in _tree:
        if type(branch) != tuple:
            for leave in branch:
                word, tag = leave
                element = element + word + ' '
    
    return traductor_en.translate(element.strip())
    
    

In [9]:
def get_wiki_info(_input):
    result = ''
    try:
        result = ''.join(wikipedia.page(_input).content.split('\n')[:2])
        result = traductor_es.translate(result)
    except:
        result = 'Lo lamento, no tengo información sobre este tema'
    return result

In [10]:
def get_position(_wiki_url):
    headers ={
        'User-Agent': 'Mozilla/5.0'
    }
    wiki_html = Request(_wiki_url, headers=headers)
    raw_html = urlopen(wiki_html).read()
    soup = BeautifulSoup(raw_html, 'html.parser')
    coordinates = []
    for pos in soup.find_all('span', attrs={'class':'geo-dms'}):

        for elm in pos.find_all('span'):
            coordinates.append(elm.text.strip())

    lat_lon = [round(parse(coordinates[0]), 2), round(parse(coordinates[1]), 2)]
    return lat_lon

In [11]:
def get_weather_api(_position):
    
    api_url = "https://api.open-meteo.com/v1/forecast"
    api_params = {
        "latitude": _position[0],
        "longitude":_position[1],
        "daily": ['temperature_2m_max','temperature_2m_min','rain_sum','windspeed_10m_max'],
        "timezone": "Europe/Berlin"
    }
    
    req = requests.get(api_url, params=api_params).json()
    return req['daily']

In [12]:
def hablar_weather(_data):
    
    frase = f"""
        Lucho,
        
        el tiempo previsto para {_data['fecha']} en {_data['ciudad']} es de una temperatura máxima de {_data['tiempo']['tmax']} grados con una mínima de {_data['tiempo']['tmin']} grados
    """
    
    return frase

In [13]:
engine = pyttsx3.init()
def speak(_texto):
    engine.setProperty('voice', 'spanish')
    engine.setProperty('rate', 150)
    engine.say(_texto)
    engine.runAndWait()

In [14]:
def skill_weather(_input):
    data = extrae_tiempo(parser.parse(_input))
    url_wiki = wikipedia.page(data['Lugar']).url
    position = get_position(url_wiki)
    weather = get_weather_api(position)
    
    if data['Fecha'] == 'hoy':
        spos = 0
    elif data['Fecha'] == 'mañana':
        spos = 1
        
    result = {
        "fecha": data['Fecha'] + ' dia ' + weather['time'][spos].split('-')[2],
        "tiempo": {
            "tmax": weather['temperature_2m_max'][spos],
            "tmin": weather['temperature_2m_min'][spos],
            "lluvia": weather['rain_sum'][spos],
            "viento": weather['windspeed_10m_max'][spos],
        },
        "ciudad": data['Lugar']
    }
    
    return hablar_weather(result)
    
    

In [15]:
def skill_knowledge(_input):
    
    tree = parser_knowledge.parse(_input)
    data = extrae_knowledge(tree)
    wikidata = get_wiki_info(data)
    
    frase = 'Lucho, esta es la información que he encontrado en wikipedia: ' + wikidata
    
    return frase
    
    

In [16]:
# def skill_openai(_input):
    
#     prompt = f"""
#         Contesta con Tono muy sarcástico a {_input}:
#     """
    
#     response = openai.Completion.create(
#       model="text-davinci-003",
#       prompt= prompt,
#       temperature=0.8,
#       max_tokens=250,
#       top_p=1.0,
#       frequency_penalty=0.6,
#       presence_penalty=0.0
#     )
    
#     return response['choices'][0]['text']

In [17]:
test_corpus = [
    "alexa que sabes sobre madrid",
    "alexa dime el tiempo de madrid para mañana",
    "alexa que tiempo va a hacer en madrid mañana"
]

## Main Function for Our Alexa Chatbot

Here you have the main function that powers our Alexa chatbot. This function serves as the central hub for the chatbot, orchestrating its various capabilities. 

By tracing back each funcion that is being called within this main function by its corresponding name, you can get a comprehensive understanding of the chatbot's inner workings.

By understanding how each function contributes to the overall behavior of the chatbot, you can extend, debug, or modify the system more effectively.


In [18]:
def Alexa(my_input):
    input_text = my_input

    print('Has dicho: ', input_text)
    print('---------------')

    tokens = word_tokenize(input_text, 'spanish')
    tokens_tags = hmm.tag(tokens)
    
    if tokens[2] == 'sabes':
        path = 'knowledge'
    elif tokens[2] == 'tiempo' or tokens[3]=='tiempo':
        path= 'weather'
    else:
        path= 'other'
    
    if path == 'weather':
        frase_speak = skill_weather(tokens_tags)
    elif path == 'knowledge':
        frase_speak = skill_knowledge(tokens_tags)
        #frase_speak = ''
    # elif path == 'other':
    #     frase_speak = skill_openai(input_text)
    

    print(frase_speak)
    speak(frase_speak)
    

In [19]:
my_input = 'Alexa dime el tiempo de barcelona para mañana'
Alexa(my_input)

Has dicho:  Alexa dime el tiempo de barcelona para mañana
---------------

        Lucho,
        
        el tiempo previsto para mañana dia 08 en barcelona es de una temperatura máxima de 25.5 grados con una mínima de 21.0 grados
    


In [21]:
my_input = 'Alexa que sabes sobre Chapadmalal'
Alexa(my_input)

Has dicho:  Alexa que sabes sobre Chapadmalal
---------------
Lucho, esta es la información que he encontrado en wikipedia: El recinto de Chapadmalal es un retiro oficial del presidente de Argentina. Está ubicado en la costa sur de Mar del Plata, en la Provincia de Buenos Aires, y sirve como residencia de verano. Tiene un complejo hotelero cercano. El recinto fue construido en 1947, durante la presidencia de Juan Perón. Raúl Alfonsín sólo lo utilizó en ocasiones limitadas. Carlos Menem ordenó varias mejoras, como un muelle de pesca, una piscina más alta y un santuario. Fernando de la Rúa lo utilizó para las entrevistas. Durante la crisis argentina de 2001, Adolfo Rodríguez Saá convocó a la misma una cumbre de gobernadores a la que asistieron sólo cinco; esta falta de apoyo provocó su dimisión. Néstor y Cristina Kirchner no lo utilizaron, sólo lo visitaron en ocasiones sus hijos. Mauricio Macri lo utilizó para una reunión de su gabinete en un ambiente menos formal.


In [36]:
# If you want to record audio voice to ask Alexa
my_input = audio_input()
Alexa(my_input)

Has dicho:  Alexa dime el tiempo de London para hoy
---------------

        Lucho,
        
        el tiempo previsto para hoy dia 07 en London es de una temperatura máxima de 29.5 grados con una mínima de 17.6 grados
    


Funny, right?

<3

Voices can be improved:

https://cloud.google.com/text-to-speech?hl=es

Google Cloud's Text-to-Speech service is indeed a powerful tool for generating high-quality voice output. The platform offers a range of voice options and fine-grained controls over aspects like pitch, speed, and emphasis. While the basic functionality can be used for free within certain usage limits, more advanced features often require a paid subscription.

The API is well-documented, and Google provides client libraries in multiple languages, including Python, to make it easy to integrate text-to-speech capabilities into your applications.